# Introduction to posless (bxibase > 6.x)

Version 6.x of bxi.base.parserconf supports command-based configuration files. That is, a configuration file can be specific to a given command. When launching a new command say `bxi-foo`, a configuration file for that command is looked for. The location file depends on various parameters described below:

0. If a file is specified with the option `--config-file`, it is directly used.
1. Otherwise, if a specific directory is specified with option `--config-dir`, it is used to find configuration files according to the policy defined below. Else, if the user is root, files are searched in `/etc`, otherwise, they are searched in `~/.config/`. 
2. Unless a specific directory is specified with option `--config-dir`, the `config_dirname` passed to `parserconf.addargs()`, is appended to the path. It defaults to `bxi`. Therefore, files are searched in `/etc/bxi/` when launched as root, or `~/.config/bxi` when launched as a normal user by default.
3. Then if a file named `bxi-foo.conf` is found, it is used for the configuration of that `bxi-foo` command.
4. Otherwise, if a `domain_name` say `bar` has been given to the `parserconf.addargs()`, and if a file named `bar.conf` exists in that path, it is used for that `bxi-foo` command.
5. Otherwise, if a file named `default.conf` is found, it is used for that `bxi-foo` command.

The `bxibase` package provides the `bxilog-posless.py` command in the documentation examples directory.
Its content is rather simple:

In [26]:
%%bash

cat ../../../packaged/doc/examples/bxilog-posless.py 

#!/usr/bin/env python

import sys
import bxi.base.log as bxilog
import bxi.base.posless as posless
import bxi.base.parserconf as bxiparserconf

if __name__ == '__main__':
    parser = posless.ArgumentParser(description='BXI Log Posless Example',
                                   formatter_class=bxiparserconf.FilteredHelpFormatter)

    
    default = bxiparserconf.getdefaultvalue(parser, 'My Section', 'variable', None,
                                            'Default_Value')
    parser.add_argument('-v', '--variable', default=default,
                        envvar='VAR',
                        help="Do something with variable. Current: %(default)s.")
    
    bxiparserconf.addargs(parser)
    args = parser.parse_args()
    
    bxilog.out("Value of variable: %s", args.variable)
        
    one = bxilog.getLogger('one.logger')
    other = bxilog.getLogger('another.logger')
    history = bxilog.getLogger('history') 
    
    # This log will go only to the history according to the

First let's have a look at its usage:

In [18]:
%%bash 

../../../packaged/doc/examples/bxilog-posless.py --help

usage: bxilog-posless.py [-h] [-v VARIABLE] [--config-dir DIR] [-C FILE] [--help-logs] [--loglevels] [--output-default-logcfg]
                         [--logcfgfile logcfgfile] [-l log-console-filters] [--log-file-filters log-file-filters]
                         [--log-file-path PATH] [--help-full]

BXI Log Posless Example

optional arguments:
  -h, --help            show this help message and exit
  -v VARIABLE, --variable VARIABLE
                        Do something with variable. Current: Default_Value.
  --help-logs           show detailed logging options and exit
  --help-full           show all options and exit

File Based Configuration:
  --config-dir DIR      The directory where the configuration file must be looked for. Value: /home/vigneras/.config/bxi.
                        Environment variable: BXICONFIGDIR


Three things must be noticed:

1. it uses a configuration directory (`--config-dir`): this is provided by parserconf module; most command will provide this option. This is where common variables used by multiple commands can be factorized. More on that soon.
2. It provides a `--variable` option. This is just an example. We will see how its content depends on the configuration file.
3. It also provides logging configuration `--help-logs`. More on that later.

# Command Line Variables Configuration

When the bxiparserconf is used, a configuration variable can be given its value according to the following increasing policy:

1. a default value;
1. a configuration file;
2. an environment variable;
3. from the command line.

The default value is given in the code itself when nothing else has been found. However if a configuration file is found, the variable is taken from it. In our example, if we create a default configuration file:

In [71]:
import os
import tempfile

DIR=tempfile.mkdtemp()

fileconf = os.path.join(DIR, 'default.conf')

with open(fileconf, 'w') as f:
    f.write("""
[My Section]
    variable = 'foo'
""")

print("File %s created" % fileconf)
    
!../../../packaged/doc/examples/bxilog-posless --config-dir {DIR} --help

File /tmp/.private/vigneras/tmpxlk5k5/default.conf created
usage: bxilog-posless [-h] [--config-dir DIR] [-C FILE] [--help-logs] [--loglevels] [--output-default-logcfg]
                      [--logcfgfile logcfgfile] [-l log-console-filters] [--log-file-filters log-file-filters]
                      [--log-file-path PATH] [--help-full] [-v VARIABLE]

BXI Log Posless Example

optional arguments:
  -h, --help            show this help message and exit
  --help-logs           show detailed logging options and exit
  --help-full           show all options and exit
  -v VARIABLE, --variable VARIABLE
                        Do something with variable. Current: foo.

File Based Configuration:
  --config-dir DIR      The directory where the configuration file must be looked for. Value:
                        /tmp/.private/vigneras/tmpxlk5k5. Environment variable: BXICONFIGDIR


Since the `default.conf` configuration file has been found, the value defined for `variable` is used and shown in the help. However, this value can be overridden by a new configuration file. Since the domain of the `bxilog-posless` command is specified (in the code) to be `bar`, the following configuration file will be used instead:

In [73]:
fileconf = os.path.join(DIR, 'bar.conf')

with open(fileconf, 'w') as f:
    f.write("""
[My Section]
    variable = 'bar'
""")

print("File %s created" % fileconf)
    
!../../../packaged/doc/examples/bxilog-posless --config-dir {DIR} --help

File /tmp/.private/vigneras/tmpxlk5k5/bar.conf created
usage: bxilog-posless [-h] [--config-dir DIR] [-C FILE] [--help-logs] [--loglevels] [--output-default-logcfg]
                      [--logcfgfile logcfgfile] [-l log-console-filters] [--log-file-filters log-file-filters]
                      [--log-file-path PATH] [--help-full] [-v VARIABLE]

BXI Log Posless Example

optional arguments:
  -h, --help            show this help message and exit
  --help-logs           show detailed logging options and exit
  --help-full           show all options and exit
  -v VARIABLE, --variable VARIABLE
                        Do something with variable. Current: bar.

File Based Configuration:
  --config-dir DIR      The directory where the configuration file must be looked for. Value:
                        /tmp/.private/vigneras/tmpxlk5k5. Environment variable: BXICONFIGDIR


Finally, one step further is to create a configuration with named `bxilog-posless.conf` with a specific configuration that overrides previous ones:

In [ ]:
fileconf = os.path.join(DIR, 'bxilog-posless.conf')

with open(fileconf, 'w') as f:
    f.write("""
[My Section]
    variable = 'baz'
""")

print("File %s created" % fileconf)
    
!../../../packaged/doc/examples/bxilog-posless --config-dir {DIR} --help

As we have seen, a specific configuration directory can be specified. A specific configuration file can also be specified. In this case, it is used directly:

In [78]:
fileconf = os.path.join(DIR, 'specific.conf')

with open(fileconf, 'w') as f:
    f.write("""
[My Section]
    variable = '123'
""")

print("File %s created" % fileconf)
    
!../../../packaged/doc/examples/bxilog-posless --config-file {fileconf} --help

File /tmp/.private/vigneras/tmpxlk5k5/specific.conf created
usage: bxilog-posless [-h] [--config-dir DIR] [-C FILE] [--help-logs] [--loglevels] [--output-default-logcfg]
                      [--logcfgfile logcfgfile] [-l log-console-filters] [--log-file-filters log-file-filters]
                      [--log-file-path PATH] [--help-full] [-v VARIABLE]

BXI Log Posless Example

optional arguments:
  -h, --help            show this help message and exit
  --help-logs           show detailed logging options and exit
  --help-full           show all options and exit
  -v VARIABLE, --variable VARIABLE
                        Do something with variable. Current: 'xyz'.

File Based Configuration:
  --config-dir DIR      The directory where the configuration file must be looked for. Value: /home/vigneras/.config/bxi.
                        Environment variable: BXICONFIGDIR


Of course the environment variable is more specific that configuration files:

In [75]:
%env VAR='xyz'

!../../../packaged/doc/examples/bxilog-posless --config-dir {DIR} --help

env: VAR='xyz'
usage: bxilog-posless [-h] [--config-dir DIR] [-C FILE] [--help-logs] [--loglevels] [--output-default-logcfg]
                      [--logcfgfile logcfgfile] [-l log-console-filters] [--log-file-filters log-file-filters]
                      [--log-file-path PATH] [--help-full] [-v VARIABLE]

BXI Log Posless Example

optional arguments:
  -h, --help            show this help message and exit
  --help-logs           show detailed logging options and exit
  --help-full           show all options and exit
  -v VARIABLE, --variable VARIABLE
                        Do something with variable. Current: 'xyz'.

File Based Configuration:
  --config-dir DIR      The directory where the configuration file must be looked for. Value:
                        /tmp/.private/vigneras/tmpxlk5k5. Environment variable: BXICONFIGDIR


And finally, the command line is even more specific:

In [76]:
!../../../packaged/doc/examples/bxilog-posless --config-dir {DIR} --variable=abc --help

usage: bxilog-posless [-h] [--config-dir DIR] [-C FILE] [--help-logs] [--loglevels] [--output-default-logcfg]
                      [--logcfgfile logcfgfile] [-l log-console-filters] [--log-file-filters log-file-filters]
                      [--log-file-path PATH] [--help-full] [-v VARIABLE]

BXI Log Posless Example

optional arguments:
  -h, --help            show this help message and exit
  --help-logs           show detailed logging options and exit
  --help-full           show all options and exit
  -v VARIABLE, --variable VARIABLE
                        Do something with variable. Current: abc.

File Based Configuration:
  --config-dir DIR      The directory where the configuration file must be looked for. Value:
                        /tmp/.private/vigneras/tmpxlk5k5. Environment variable: BXICONFIGDIR


# Configuration Files Inclusion

The `bxi.base.parserconf` module implements configuration file inclusion: one file can include another. When file `F` includes file `I`, all variables defined in `I` are also defined in `F`, and their values are the one in `I` unless they are overridden by `F`. Consider the following example:

In [91]:
fileconf = os.path.join(DIR, 'I.conf')

with open(fileconf, 'w') as f:
    f.write("""
[My Section]
    variable = 'defined_in_I'
    stuff = 'defined_in_I'
""")

print("File %s created" % fileconf)

fileconf = os.path.join(DIR, 'F.conf')

with open(fileconf, 'w') as f:
    f.write("""
includes = I.conf,
[My Section]
    variable = 'defined_in_F'
""")

print("File %s created" % fileconf)

    
!../../../packaged/doc/examples/bxilog-posless --config-file {fileconf} --help

env: -d=VAR
File /tmp/.private/vigneras/tmpxlk5k5/I.conf created
File /tmp/.private/vigneras/tmpxlk5k5/F.conf created
usage: bxilog-posless [-h] [--config-dir DIR] [-C FILE] [--help-logs] [--loglevels] [--output-default-logcfg]
                      [--logcfgfile logcfgfile] [-l log-console-filters] [--log-file-filters log-file-filters]
                      [--log-file-path PATH] [--help-full] [-v VARIABLE] [-stuff STUFF]

BXI Log Posless Example

optional arguments:
  -h, --help            show this help message and exit
  --help-logs           show detailed logging options and exit
  --help-full           show all options and exit
  -v VARIABLE, --variable VARIABLE
                        Do something with variable. Current: 'xyz'.
  -stuff STUFF, --stuff STUFF
                        Do something with stuff. Current: defined_in_I.

File Based Configuration:
  --config-dir DIR      The directory where the configuration file must be looked for. Value: /home/vigneras/.config/bxi.
    

# Logging Configuration Files

The `bxi.base.parserconf` module implements a special policy for logging configuration: it looks after a variable called `bxilog.default.configfile` and if found, use it as a path towards the logging configuration file.